In [1]:
import numpy as np

test = [1, 2, 3, 4, 5, 6, 7, 8, 9]

print(test)
print(len(test))

test.remove(9)
print(test)
print(len(test))

[1, 2, 3, 4, 5, 6, 7, 8, 9]
9
[1, 2, 3, 4, 5, 6, 7, 8]
8


In [6]:
import tkinter as tk

def write_to_entry():
    value_to_write = "Hello, Tkinter!"
    entry_widget.delete(0, tk.END)  # Efface le contenu existant de l'Entry
    entry_widget.insert(0, value_to_write)  # Insère la nouvelle valeur

# Crée une fenêtre Tkinter
root = tk.Tk()
root.title("Exemple d'Entry")

# Crée un widget Entry
entry_widget = tk.Entry(root, width=30)
entry_widget.grid(row=0, column=0, padx=10, pady=10)

# Crée un bouton pour écrire dans l'Entry
button = tk.Button(root, text="Écrire dans l'Entry", command=write_to_entry)
button.grid(row=1, column=0, pady=5)

# Lance la boucle principale de la fenêtre
root.mainloop()
